In [ ]:
import numpy as np
from pathlib import Path
import sarxarray
import dask

## Data loading

In [ ]:
# Path to the interferogram dataset
path = Path('./nl_amsterdam_s1_asc_t088')

# Make a list of interferograms to read
list_ifgs = [p for p in path.rglob('*_cint_srd.raw')]
list_ifgs.sort()
list_ifgs

In [ ]:
# Metadata of the stack, assume known.
shape=(2000, 4000)

In [ ]:
# Define reading chunks
reading_chunks = (1000,1000)

In [ ]:
sarxarray.from_binary?

In [ ]:
stack = sarxarray.from_binary(list_ifgs, shape, dtype=np.complex64, chunks=reading_chunks)
stack

In [ ]:
dask.visualize(stack['amplitude'])

## Update space and time coordinates

In [ ]:
# Geo-coordinates
f_lat = [path / 'lat.raw']
f_lon = [path / 'lon.raw']

In [ ]:
# Load coordinates
lat = sarxarray.from_binary(
    f_lat, shape, vlabel="lat", dtype=np.float32, chunks=reading_chunks
)
lon = sarxarray.from_binary(
    f_lon, shape, vlabel="lon", dtype=np.float32, chunks=reading_chunks
)
stack = stack.assign_coords(
    lat=(("azimuth", "range"), lat.squeeze().lat.data),
    lon=(("azimuth", "range"), lon.squeeze().lon.data),
)

In [ ]:
stack

In [ ]:
# Update the stack with the time coordinates
time_coords = [np.datetime64(f"{t.stem[0:4]}-{t.stem[4:6]}-{t.stem[6:8]}") for t in list_ifgs]
time_coords = np.array(time_coords, dtype='datetime64[ns]')
time_coords


In [ ]:
stack = stack.assign_coords(time=("time", time_coords))
stack

## Multi-Looking

In [ ]:
stack_multilook = stack.slcstack.multi_look((1,2))
stack_multilook

## Mean-Reflection-Map (MRM) 

In [ ]:
mrm = stack_multilook.slcstack.mrm()
mrm

In [ ]:
dask.visualize(mrm)

In [ ]:
# Visualize
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
fig.set_size_inches((5,5))
im = mrm.plot(ax=ax, robust=True, cmap='gray')
im.set_clim([0, 50000])

## Subset in space and time

In [ ]:
# Indexing
stack_subset_space1 = stack_multilook.isel(azimuth=slice(1000, 2000), range=slice(1000, 2000))
stack_subset_space1

In [ ]:
print(stack_multilook['lat'].max().values)
print(stack_multilook['lat'].min().values)
print(stack_multilook['lon'].max().values)
print(stack_multilook['lon'].min().values)

In [ ]:
# Value based
# stack_subset_space1 = stack.where((stack['lat'] > 52.4), drop=True) # This does not work becasue 2d
stack_subset_space2 = stack_multilook.where((stack_multilook['lat'] > 52.3)).compute() # This works
stack_subset_space2

In [ ]:
mrm_subset = stack_subset_space2.slcstack.mrm()

In [ ]:
mrm_subset

In [ ]:
# Visualize
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
fig.set_size_inches((5,5))
im = mrm_subset.plot(ax=ax, robust=True, cmap='gray')
im.set_clim([0, 50000])

In [ ]:
# Value based time subset
stack_subset_time1 = stack.where((stack['time'] > np.datetime64("2018-10-14")), drop=True) # This works because 1d
stack_subset_time1

## Exercise: temporal and spatial subset (10 mins)
1. Select this region: lon [52.3, 52.4], lat [4.8, 5.0]
2. Select all interferograms from after 2018-10-26
3. Visualize the MRM of the selected stack

In [ ]:
stack_subset1 = stack_multilook.where((stack_multilook['time'] > np.datetime64("2018-10-26")), drop=True)
stack_subset1 = stack_subset1.where((stack_subset1['lat'] > 52.3) & (stack_subset1['lat'] < 52.4) & (stack_subset1['lon'] > 4.8) & (stack_subset1['lon'] < 5.0))
mrm_subset1 = stack_subset1.slcstack.mrm()

# Visualize
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
fig.set_size_inches((5,5))
im = mrm_subset1.plot(ax=ax, robust=True, cmap='gray')
im.set_clim([0, 50000])